# Sub functions 

In [1]:
from ipynb.fs.full.GlobalVars import *
from ipynb.fs.full.Jacobian import *

In [2]:
"""
    Create the function f wich correspond to this equality : du/dt = expr(u, v). 
    Here du/dt -> un - un-1 / delta
    
    @return: the new function ready to be evaluated.
    
    @var: the symbol we want to evalute (u or v)
    @term_1: the previous term
    @expr: the function tied.
    @use_trapezoidal: boolean to check if we use trapezoidal method or backward euler one.
"""

def create_f(var, term_1, expr, use_trapezoidal, velocity_n):
    
    #In this 2 cases we want to have f(x) = 0 where f is the evaluation function.
    #Trapezoidal method : d_un+1 = 2(un+1 - un) / h - d_un
    if(use_trapezoidal):
        
        #term_1_evaluated = expr.subs(var, term_1)
        return (2/delta)*(var - term_1) - 2*expr
    
    #Backward Euler method : f = (un - un_1)/h - d_un = 0
    else:
        return (var - term_1)/delta - velocity_n - expr

In [3]:
"""
    Same as create_f. The only difference is that this time we 
    keep it in a vector cause we have two different functions.
    
    @return: a vector containing the 2 new functions ready to be evaluated.
    
    @xn_1_global: previous term
    @f1: first function
    @f2 second function
"""

def create_2_f(xn_1_global, f1, f2, use_trapezoidal, velocity_n):
    return np.array([create_f(u, xn_1_global[0], f1, use_trapezoidal, velocity_n[0]), create_f(v, xn_1_global[1], f2, use_trapezoidal, velocity_n[1])])

In [4]:
"""
    Create the 2x2 matrix wich correspond to the derivate of F(xk) 
    when xk is vector like xk = (f1, f2).
    This matrix = [df1/du , df2/du]
                  [df1/dv , df2/dv]
    
    @return: new matrix created.
    
    @xk_1: previous term
    @f1: first function
    @f2: second function
"""

def f_prime_2_eqs(xk_1, f1, f2):
    res = np.array([[0., 0.], [0., 0.]])
    symbols = [u, v]
    tab_f = [f1, f2]
    
    for i in range(2):
        for j in range(2):
            res[i][j] = Jacobian_square_mat(symbols[i], tab_f[j]).subs(symbols[i], xk_1[j])
    res[0][0]*=2
    res[1][1]*=2
    return res

In [5]:
"""
    Create the term F(xk_1) wich is a vector of size 2 containing the evaluation
    respectivly of f1 and f2.
    
    @return: the term F(xk_1)
    
    @xk_1 previous term
    @vec_f: list containing f1 and f2
"""

def create_fxk_1(xk_1, vec_f):
    res = np.array([0., 0.])
    for i in range(2):
        
        res[i] = vec_f[i].evalf(subs={u: xk_1[0], v: xk_1[1]})
    
    return res

In [6]:
"""
    Create a list containings the lists of points to build the u function
    and the v function + x and y variables if we need it.
    
    @return: [ [ [t0, u0], [t1, u1], ..., [tn, un] ] , [ [t0, v0], [t1, v1], ..., [tn, vn] ] ] 
    + x and y points's list
    
    @time: list containing t0, t1, t2, ...
    @fu: list containing u0, u1, u2, ...
    @fv: list containing v0, v1, v2, ...
    @x: containing all the x calculated
    @y: containing all the y calculated
"""

def result_solver(time, fu, fv=[], x=[], y=[]):
    final_points_u = []
    
    for i in range(len(fu)):
        final_points_u.append([time[i], fu[i]])
    
    #Only one equation
    if(fv == []):
        return final_points_u
    
    #2 equations
    else:
        final_points_v = []
    
        for i in range(len(fu)):
            final_points_v.append([time[i], fv[i]])
    
    # 2 equations for sure so we stop here
    if(x == []):
        return [final_points_u, final_points_v]
    
    #4 equations
    else:
        final_points_x = []
        final_points_y = []
        
        for i in range(len(fu)):
            final_points_x.append([time[i], x[i]])
            final_points_y.append([time[i], y[i]])
    
    return [final_points_u, final_points_v, final_points_x, final_points_y]